# Цель задания

Собрать новый датасет с помощью парсинга данных.

## Формат сдачи

Пришлите ссылку на репозиторий, в котором находятся:
* Jupiter Notebook с кодом.
* Итоговый датасет — файл в формате .csv.

**Критерии оценки**:

* Датасет содержит все необходимые поля; размер датасета соответствует эталонному на 90% и более, обучена модель, отправлен сабмишн и выводы — 5 баллов.
* Датасет содержит все необходимые поля; размер датасета соответствует эталонному менее чем на 90%, правильная логика на этапах матчинга, парсинга доп. характеристик авто и мерджа с исходным датасетом — 4 балла.
* Датасет содержит все или почти все необходимые поля; размер датасета соответствует эталонному менее чем на 90% допущена серьезная ошибка на одном из этапов: матчинг, парсинг доп. характеристик авто и мердж с исходным датасетом — 3 балла.
* Получилось спарсить ссылки на модели автомобилей — 2 балла.
* Код не исполняется; нет датасета — 1 балл.

# <center> 🤼‍♀️ Разминаемся (Задание 1)
В качестве первого задания вам предстоит достать значения средних зарплат по городам России. Сайт, на котором они хранятся: https://stepik.org/media/attachments/lesson/866758/mean_salary_by_city.html

Ответ - датафрейм, котором города идут в алфавитном порядке.

Пример ответа:

<left> <img src='https://github.com/PeMikj/images/blob/main/images/image1.png?raw=true' width="550" >

In [183]:
from lxml import etree, html as lhtml
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from lcs2 import diff, diff_ranges, lcs, lcs_indices, lcs_length
import numpy as np
import re
from tqdm.autonotebook import tqdm
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import os
import joblib

pd.set_option('display.max_columns', None)

In [5]:
url = 'https://stepik.org/media/attachments/lesson/866758/mean_salary_by_city.html'
response = requests.get(url)
content = response.content

tree = lhtml.fromstring(content)

In [6]:
salary_info = {
    'city': tree.xpath('//div[starts-with(@class, "reg_name")]/a/text()'),
    'mean_salary': tree.xpath('//div/span[starts-with(@class, "reg_salary")]/text()'),
}

df = pd.DataFrame(salary_info, index=None)

def clean_string(s):
    s =  ''.join(s.split()) if isinstance(s, str) else s
    s = s[:-1]
    return s

# Применение функции ко всем строкам всех столбцов
df['mean_salary'] = df['mean_salary'].map(clean_string)
df['mean_salary'] = df['mean_salary'].astype('int')
df.head()

,city,mean_salary
0,Анадырь,129200
1,Москва,113600
2,Салехард,106400
3,Южно-Сахалинск,99000
4,Магадан,95200


# <center> 🌍 Парсим [automobili.ru](https://automobili.ru/cars/catalog/) 🚗

В этой задаче вам предстоит спарсить дополнительные данные по моделям машин с сайта: https://automobili.ru/cars/catalog/ и добавить их в существующий датасет. Для того чтобы было проще понимать, в каком месте вы ошибаетесь (если такое происходит), мы разбили эту задачу на несколько степов, где постепенно будем парсить данный сайт.

<left> <img src='https://github.com/PeMikj/images/blob/main/images/image2.png?raw=true' width="750" >

## <center> 🕊 Собираем ссылки 🔗

Для того чтобы начать доставать информацию о моделях, нужно вначале получить ссылки на эти модели. На первой странице сайта находится список всех марок. Вам нужно:

1) Собрать все ссылки на автомобильные бренды (средствами автоматического парсинга).

2) Пройтись по каждой из полученных ссылок и спарсить названия конкретных моделей автомобилей.

<left> <img src='https://github.com/PeMikj/images/blob/main/images/image3.png?raw=true' width="750" >
<left> <img src='https://github.com/PeMikj/images/blob/main/images/image4.png?raw=true' width="750" >

В итоге у вас должно найтись 325 ссылок, которые выглядят примерно так:

<left> <img src='https://github.com/PeMikj/images/blob/main/images/image5.png?raw=true' width="750" >

Ответ - датафрейм с ссылками и названием модели. Датафрейм должен быть отсортирован по названию модели, а затем по ссылкам.
`df.sort_values(by=['model', 'link'])`

Пример:

<left> <img src='https://github.com/PeMikj/images/blob/main/images/image6.png?raw=true' width="750" >

In [7]:
# URL страницы с автомобильными брендами
url = 'https://automobili.ru/cars/catalog/'

# Отправляем запрос к странице
response = requests.get(url)

# Проверяем, что запрос успешен
if response.status_code == 200:
    # Создаем объект BeautifulSoup для парсинга HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    links = soup.find_all('a', class_="car-brand-link")
    
    brand_links = [urljoin(url, link['href']) for link in links]

else:
    print(f'Ошибка при запросе страницы: {response.status_code}')
    
models_list = []
models_links = []
temp_list = []

for link in brand_links:
    # comment: 
    response = requests.get(link)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')    
        model_links = soup.find_all('a', class_="model-item__title")
        brand = soup.find('li', class_="disabled").get_text(strip=True)
        
        for link in model_links:
            # comment: 
            # model_txt = link['href'][14:].replace('/', ' ').strip()
            # temp_list.append([urljoin(url, link['href']), model_txt])
            temp_list.append([urljoin(url, link['href']), brand + ' ' + link.get_text(strip=True)])
            
    else:
        print(f'Ошибка при запросе страницы: {response.status_code}')
# end for

models_df = pd.DataFrame(temp_list, columns=['link', 'model'])

models_df.sort_values(by=['link', 'model'])

,link,model
1,https://automobili.ru/cars/catalog/aston_marti...,Aston Martin DB11
2,https://automobili.ru/cars/catalog/aston_marti...,Aston Martin DBS Superleggera
0,https://automobili.ru/cars/catalog/aston_marti...,Aston Martin Vantage
14,https://automobili.ru/cars/catalog/audi/Q5/,Audi Q5
3,https://automobili.ru/cars/catalog/audi/a3/,Audi A3
...,...,...
314,https://automobili.ru/cars/catalog/volvo/v90-c...,Volvo V90 Cross Country
315,https://automobili.ru/cars/catalog/volvo/xc40/,Volvo XC40
316,https://automobili.ru/cars/catalog/volvo/xc60/,Volvo XC60
318,https://automobili.ru/cars/catalog/volvo/xc90-...,Volvo XC90 TwinEngine


In [8]:
models_df

,link,model
0,https://automobili.ru/cars/catalog/aston_marti...,Aston Martin Vantage
1,https://automobili.ru/cars/catalog/aston_marti...,Aston Martin DB11
2,https://automobili.ru/cars/catalog/aston_marti...,Aston Martin DBS Superleggera
3,https://automobili.ru/cars/catalog/audi/a3/,Audi A3
4,https://automobili.ru/cars/catalog/audi/a4/,Audi A4
...,...,...
320,https://automobili.ru/cars/catalog/gas/1994/,ГАЗ ГАЗель Бизнес
321,https://automobili.ru/cars/catalog/uaz/hunter/,УАЗ Хантер
322,https://automobili.ru/cars/catalog/uaz/patriot/,УАЗ Патриот
323,https://automobili.ru/cars/catalog/uaz/pickup/,УАЗ Пикап


## <center> 👉👈 Матчим ссылки с датасетом

Отлично! Мы получили ссылки на все модели машин, которые есть на сайте, но нам понадобится только небольшая часть, так как многие модели отсутствуют в изначальном датасете. В этом задании вам предстоит сопоставить ссылки и машины из датасета `quickstart_train.csv`.

In [9]:
import pandas as pd

path = 'https://stepik.org/media/attachments/lesson/866758/quickstart_train.csv'

df = pd.read_csv(path)

df['model'] = df['model'].map(lambda x: x.replace('VW', 'Volkswagen'))

df.head(3)

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,109.99,another_bug,4.737759,1.214131e+07,0.1,180.855726,0.023174,174,170
1,O41613818T,Volkswagen Polo VI,economy,petrol,3.90,2015,78218,2021,34.48,electro_bug,4.480517,1.803909e+07,0.0,187.862734,12.306011,174,174
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,34.93,gear_stick,4.768391,1.588366e+07,0.1,102.382857,2.513319,174,173


<left> <img src='https://github.com/PeMikj/images/blob/main/images/image7.png?raw=true' width="750" >
<left> <img src='https://github.com/PeMikj/images/blob/main/images/image8.png?raw=true' width="300" >



Далее нам нужно сматчить названия моделей машин в исходном и спаршенном датафреймах.
Для решения задачи можно использовать разные способы измерения близости между двумя строками.

Мы будем использовать алгоритм  нахождения наибольшей общей подпоследовательности - [википедия.](https://ru.wikipedia.org/wiki/%D0%9D%D0%B0%D0%B8%D0%B1%D0%BE%D0%BB%D1%8C%D1%88%D0%B0%D1%8F_%D0%BE%D0%B1%D1%89%D0%B0%D1%8F_%D0%BF%D0%BE%D0%B4%D0%BF%D0%BE%D1%81%D0%BB%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D1%8C)

Для каждой модели в нашем исходном датафрейме нужно:

1) Найти строку с названием модели в напаршенных данных с максимальным значением наибольшей общей подпоследовательности.

2) Нормализовать значение наибольшей общей подпоследовательности на длину строки в исходном датафрейме.

3) Отсечь те случаи, где нормализованное значение меньше 0.85.

4) Если не удалось найти матч - заполняем np.nan.


P.S. Также нужно заменить `vw` на `volkswagen` в изначальном датасете, чтоб было больше совпадений. Нужно понимать, что идеально сматчить не всегда получится, поэтому иногда приходится прибегать к эвристикам, но мы в данном задании этого делать не будем.

Пример матчинга между названиями машин в исходном датафрейме и напаршенном.
Для каждого названия авто из исходного датафрейма находится название из напаршенного с наибольшим значением наибольшей общей подпоследовательности.

<left> <img src='https://github.com/PeMikj/images/blob/main/images/image9.png?raw=true' width="550" >

In [10]:
# %%time
lcs_list = []
lcs_models = []

for df_model in df['model']:
    # comment: 
    lcs_list = []
    for index, model in models_df.iterrows():
        # comment: 
        lcs = lcs_length(df_model.lower(), model['model'].lower())
        lcs_koef = lcs / len(df_model)
        
        if lcs_koef >= 0.85:
            lcs_list.append(tuple([df_model, model['model'], round(lcs_koef, 2), model['link']]))
        
    # end for
    lcs_list = sorted(lcs_list, key=lambda x: x[2], reverse=True)
    if len(lcs_list):
        lcs_models.append(lcs_list[0])
    else:
        lcs_models.append([df_model, np.nan, np.nan, np.nan])
# end for

print(lcs_models)

[['Kia Rio X-line', nan, nan, nan], ['Volkswagen Polo VI', nan, nan, nan], ('Renault Sandero', 'Renault Sandero', 1.0, 'https://automobili.ru/cars/catalog/renault/sandero/'), ('Mercedes-Benz GLC', 'Mercedes-Benz GLC', 1.0, 'https://automobili.ru/cars/catalog/mercedes-benz/glc/'), ('Renault Sandero', 'Renault Sandero', 1.0, 'https://automobili.ru/cars/catalog/renault/sandero/'), ('Skoda Rapid', 'Skoda Rapid', 1.0, 'https://automobili.ru/cars/catalog/skoda/rapid/'), ('Nissan Qashqai', 'Nissan Qashqai', 1.0, 'https://automobili.ru/cars/catalog/nissan/qashqai/'), ['Tesla Model 3', nan, nan, nan], ('Kia Sportage', 'Kia Sportage', 1.0, 'https://automobili.ru/cars/catalog/kia/sportage/'), ('Smart ForFour', 'smart Forfour', 1.0, 'https://automobili.ru/cars/catalog/smart/forfour/'), ('Volkswagen Polo', 'Volkswagen Polo', 1.0, 'https://automobili.ru/cars/catalog/volkswagen/polo/'), ['Volkswagen Polo VI', nan, nan, nan], ('Kia Rio', 'Kia Rio', 1.0, 'https://automobili.ru/cars/catalog/kia/rio_new/

In [11]:
df_parsed_models = pd.DataFrame(lcs_models, columns=['model', 'parsed_model', 'lcs', 'link'])

In [29]:
# %%time
year_list = []
mod_list = []
price_list = []
engine_list = []
power_list = []
box_list = []
trans_list = []
body_list = []

for index, model in tqdm(df_parsed_models.iterrows(), total=df_parsed_models.shape[0]):
    if pd.isna(model['link']) == False:
        url = model['link']
        response = requests.get(url)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')    
            year_list.append(re.search(r'(?<=\s)\d{4}', soup.find('li', class_="disabled").get_text(strip=True)).group())
            
            mod_table = soup.find('table', class_="modifications-table modifications__table hover unstriped")
            
            tbody = mod_table.find('tbody')
            trows = tbody.find_all('tr')
            tcells = trows[0].find_all('td')         
            
            # вытащим модификацию
            mod = tcells[1].get_text()
            mod_list.append(mod)
                        
            # вытащим цену
            pattern = r'\d+'
            matches = re.findall(pattern, tcells[2].get_text())
            price = ''.join(matches)  
            price_list.append(price)

            # вытащим тип двигателя
            engine = tcells[3].get_text()
            engine_list.append(engine)
            
            # вытащим мощность в л.с.
            pattern = r'\d+'
            matches = re.findall(pattern, tcells[4].get_text())
            power = ''.join(matches) 
            power_list.append(power)
            
            # вытащим тип КПП
            box = tcells[5].get_text()
            box_list.append(box)
            
            # вытащим тип трансмиссии
            trans = tcells[6].get_text()
            trans_list.append(trans)
            
            # вытащим тип кузова
            body = tcells[7].get_text()
            body_list.append(body)
            
            _ = 1
        else:
            print(f'Ошибка при запросе страницы: {response.status_code}')
    else:
        year_list.append(np.nan)
        mod_list.append(np.nan)
        price_list.append(np.nan)
        engine_list.append(np.nan)
        power_list.append(np.nan)
        box_list.append(np.nan)
        trans_list.append(np.nan)
        body_list.append(np.nan)
# end for

df_parsed_models['year'] = year_list   
df_parsed_models['mod'] = mod_list
df_parsed_models['price'] = price_list
df_parsed_models['engine'] = engine_list
df_parsed_models['power'] = power_list
df_parsed_models['box'] = box_list  
df_parsed_models['trans'] = trans_list
df_parsed_models['body'] = body_list

  0%|          | 0/2337 [00:00<?, ?it/s]

In [106]:
df_parsed_models

,model,parsed_model,lcs,link,mod,price,engine,power,box,trans,body,year
0,Kia Rio X-line,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Volkswagen Polo VI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Renault Sandero,Renault Sandero,1.0,https://automobili.ru/cars/catalog/renault/san...,1.6 5MT Access,697000,Бензин,82,Механическая,Передний,Хэтчбек,2018
3,Mercedes-Benz GLC,Mercedes-Benz GLC,1.0,https://automobili.ru/cars/catalog/mercedes-be...,2.0 9AT Premium,4690000,Бензин,197,Классический автомат,Полный,Внедорожник,2019
4,Renault Sandero,Renault Sandero,1.0,https://automobili.ru/cars/catalog/renault/san...,1.6 5MT Access,697000,Бензин,82,Механическая,Передний,Хэтчбек,2018
...,...,...,...,...,...,...,...,...,...,...,...,...
2332,Smart ForFour,smart Forfour,1.0,https://automobili.ru/cars/catalog/smart/forfour/,1.0 5MT,890000,Бензин,71,Механическая,Задний,Хэтчбек,2014
2333,Audi A4,Audi A4,1.0,https://automobili.ru/cars/catalog/audi/a4/,35 1.4 7AMT,2125000,Бензин,150,Робот с одним сцеплением,Передний,Седан,2015
2334,Kia Rio,Kia Rio,1.0,https://automobili.ru/cars/catalog/kia/rio_new/,1.4 6МТ Classic,830900,Бензин,100,Механическая,Передний,Седан,2020
2335,Renault Sandero,Renault Sandero,1.0,https://automobili.ru/cars/catalog/renault/san...,1.6 5MT Access,697000,Бензин,82,Механическая,Передний,Хэтчбек,2018


## <center> ⚗️ Достаем технические характеристики

Достаем информацию о машинах

Отлично! Ссылки мы достали, теперь пришло время получить необходимые данные из них. Это скриншот того, как выглядит страничка сайта для конкретной модели (в нашем случае Renault Sandero):

<left> <img src='https://github.com/PeMikj/images/blob/main/images/image10.png?raw=true' width="750" >


Отсюда вам необходимо для каждой модели получить следующую информацию:

1) `year` - год начала выпуска модели (целое число);
2) `mod` - название модификации;
3) `price` - рекомендованная цена (целое число);
4) `engine` - тип двигателя;
5) `power` - мощность в л.с.;
6) `box` - тип коробки передач;
7) `trans` - тип трансмиссии;
8) `body` - тип кузова;


Это все можно сделать при помощи BeautifulSoap.

Мы берем значения только для самой первой модификации!

Далее соединяем с нашим исходным датасетом. `model` - столбик по которому мы соединяем датасеты.

In [166]:
cat_features = ['model', 
                'car_type', 
                'fuel_type', 
                'rarest_work', 
                'most_freq_work',
                'mod',
                'box',
                'trans',
                'body'
                ] # категориальные признаки

In [156]:
original_df = pd.read_csv('../data/X.csv')
y = pd.read_csv('../data/y.csv')

In [119]:
df_parsed_models.to_csv('../data/df_parsed_models.csv', sep=',', index=False, encoding='utf-8')

In [157]:
df_parsed_models_copy = pd.read_csv('../data/df_parsed_models.csv')

In [ ]:
df_parsed_models_copy

In [158]:
X = pd.merge(original_df, df_parsed_models_copy, on='model', how='left')

In [159]:
X.drop_duplicates(inplace=True)

In [160]:
X.drop(['parsed_model', 'lcs', 'link', 'engine'], axis=1, inplace=True)

In [167]:
X[cat_features] = X[cat_features]. fillna('')

In [ ]:
X

In [168]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [162]:
X.nunique()

model                    26
car_type                  4
fuel_type                 2
car_rating              269
year_to_start             7
riders                 2306
year_to_work              9
mean_rating            2223
distance_sum           2337
rating_min               91
speed_max              1938
rides_count              16
ride_time_median         70
ride_stops_count        269
user_quality_median    2337
rider_rating_mean       785
car_accidents_sum      1007
driver_expirience      2290
driver_age_mean         726
fix_duration_median      26
mean_destroy_degree    1158
rarest_work               6
most_freq_work            2
mod                      15
price                    15
power                     9
box                       3
trans                     3
body                      4
year                      7
dtype: int64

In [169]:
clf = CatBoostClassifier(thread_count=-1,
                         random_seed=42,
                         cat_features=cat_features)

clf.fit(
    X_train,
    y_train,
    eval_set=(X_test, y_test),
    verbose=200,
    use_best_model=True,
    plot=False,
    early_stopping_rounds=100,
)

Learning rate set to 0.109335
0:	learn: 1.6931931	test: 1.7038239	best: 1.7038239 (0)	total: 395ms	remaining: 6m 34s
200:	learn: 0.0377306	test: 0.0713336	best: 0.0713293 (199)	total: 57.9s	remaining: 3m 50s
400:	learn: 0.0187497	test: 0.0608555	best: 0.0608555 (400)	total: 1m 54s	remaining: 2m 50s
600:	learn: 0.0115413	test: 0.0580365	best: 0.0578489 (548)	total: 2m 49s	remaining: 1m 52s
800:	learn: 0.0082694	test: 0.0565814	best: 0.0565039 (788)	total: 3m 45s	remaining: 56.1s
999:	learn: 0.0063370	test: 0.0557978	best: 0.0557284 (998)	total: 4m 40s	remaining: 0us

bestTest = 0.05572841844
bestIteration = 998

Shrink model to first 999 iterations.


In [192]:
y_clf = clf.predict(X_test)
clf_preds = clf.predict_proba(X_test)

print(roc_auc_score(y_test['target_class'], clf_preds, multi_class='ovr'))

0.9996785947412167


In [193]:
y_pred = clf.predict(X_test)
score = accuracy_score(y_test, y_pred)

score

0.9871794871794872

In [184]:
directory = os.path.join('..', 'data', 'models')
file_path = os.path.join(directory, 'cb_parsed.pkl')

os.makedirs(directory, exist_ok=True)

joblib.dump(clf, file_path)

['../data/models/cb_parsed.pkl']

In [190]:
df_test = pd.read_csv('../data/test.csv')

In [191]:
y_submit = clf.predict(df_test)
# submission = pd.read_csv('../data/test.csv')
# submission['target_class'] = y_submit.ravel()
# submission[['car_id', 'target_class']].to_csv('../data/submission_parsed.csv', index=False)

CatBoostError: Invalid cat_features[3] = 21 value: index must be < 11.

Получившийся датасет - решение задачи (не изменяйте порядок строк в изначальном датасете).

Пример результата (для удобства представления в исходном датасете сохранены только колонки `car_id` и `model`, но вам нужны все колонки из исходного датасета):

<left> <img src='https://github.com/PeMikj/images/blob/main/images/image11.png?raw=true' width="850" >

## <center> 🏋️‍♂️Тренировка с новыми данными

Обучите модель на обогащенном датасете и сравните результат с предыдущими.

Отправьте сабмишн на kaggle.

Сделайте выводы.